### 1. To mount google drive 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 2. Home directory
Giving the pathname for the home directory

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdriv}e/My Drive/Kaggle"

In [ ]:
pwd


'/content'

### 3. Instlation of tensorflow gpu
Tensorflow-gpu is installed and nvidia-smi provides us with the version and usage details of the gpu

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!nvidia-smi

Mon Mar  8 18:49:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 4. Setting up the path 

In [ ]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


### 5. Importing data from kaggle

In [ ]:
!kaggle datasets download -d vishalmane109/indian-currency-note-images-dataset-2020

### 6. Modules
Importing the necessary modules for the whole program

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.layers import Input, Lambda,Dense,Flatten
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from glob import glob

### 7. Unzipping the imported data from kaggle

In [ ]:
!unzip \*.zip  && rm *.zip

### 8. Defining path for train and test dataset

In [ ]:
IMAGE_SIZE = [224,224]
train_path = '/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/training'
test_path = '/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/validation'

### 9. Defining inceptionV3
Loading the weights trained on imagenet data. We can have the input tensor of our choiche since we have not included the fully connected top layer. Include top is to include the fully-connected layer at the top, as the last layer of the network. 

In [ ]:
inception = InceptionV3(input_shape = IMAGE_SIZE + [3], weights ='imagenet', include_top = False)

87916544/87910968 [==============================] - 0s 0us/step


### 10. Turning trainable layers to false
We have to train only the top layers (which were randomly initialized), so freeze all convolutional InceptionV3 layers

In [ ]:
for layer in inception.layers:
    layer.trainable = False

### 10. Output entries and training folder
The following code helps to display the different training folders(or classes).


In [ ]:
folders = glob('/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/training/*')
folders

['/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/training/10',
 '/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/training/100',
 '/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/training/20',
 '/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/training/200',
 '/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/training/2000',
 '/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/training/50',
 '/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/training/500',
 '/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/training/Background']

### 11. Flattening the output layer


In [ ]:
x = Flatten()(inception.output)


### 12 Prediction
Prediction should be made on the 8 different classes(no of training folders). Since this is a multi classification problem softmax activation is used.

In [ ]:
prediction = Dense(len(folders), activation = 'softmax')(x)

### 13. Defining the model
The model is defined and summary for the model is obtained

In [ ]:

model = Model(inputs = inception.input, outputs = prediction)
print(inception.input)

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

### 14. Compiling
Model is compiled with the selected loss function and optimizer and using accuray metrics.

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

### 15. Data
Data augmentation is done and rescaled by a factor of 255 for processing. Sheared zoomed and flip are done in augmentation

In [ ]:
train_data = ImageDataGenerator(rescale = 1./255,
                               shear_range =0.2,
                               zoom_range = 0.2,
                               horizontal_flip = True)
test_data = ImageDataGenerator(rescale = 1./255)

### 16. Defining the path
Path for train and test set are provided with batch size.

In [ ]:
train_set = train_data.flow_from_directory(train_path,
                                              target_size = (224,224),
                                              batch_size =16,
                                              class_mode = 'categorical')

Found 3566 images belonging to 8 classes.


In [ ]:
test_set = test_data.flow_from_directory(test_path,
                                              target_size = (224,224),
                                              batch_size =16,
                                              class_mode = 'categorical')

Found 345 images belonging to 8 classes.


In [ ]:
!pip install pyyaml h5py

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

### 17. Training the model
The model is trained in 50 epochs in a batch size of 16. 3566/16 i.e around 223 images of 16 batches. And after the training the whole model is saved in H5 format inorder to access it later. An accuray of 90% was acquired in train data and validation accuracy was about 84%. 

In [ ]:
model.fit(train_set, validation_data = test_set,
                       epochs =50,
                       steps_per_epoch = len(train_set),
                       validation_steps = len(test_set))
model.save('currency_inception.H5')

Epoch 1/80
112/112 [==============================] - ETA: 0s - loss: 8.7469 - accuracy: 0.3597 

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


112/112 [==============================] - 1683s 15s/step - loss: 8.7168 - accuracy: 0.3608 - val_loss: 1.7271 - val_accuracy: 0.6812
Epoch 2/80
112/112 [==============================] - 267s 2s/step - loss: 2.1014 - accuracy: 0.6848 - val_loss: 1.9007 - val_accuracy: 0.7130
Epoch 3/80
112/112 [==============================] - 266s 2s/step - loss: 1.7555 - accuracy: 0.7527 - val_loss: 2.4219 - val_accuracy: 0.7043
Epoch 4/80
112/112 [==============================] - 267s 2s/step - loss: 1.4861 - accuracy: 0.7952 - val_loss: 3.0809 - val_accuracy: 0.6812
Epoch 5/80
112/112 [==============================] - 268s 2s/step - loss: 1.7349 - accuracy: 0.7913 - val_loss: 2.5584 - val_accuracy: 0.7420
Epoch 6/80
112/112 [==============================] - 268s 2s/step - loss: 1.2881 - accuracy: 0.8424 - val_loss: 2.2253 - val_accuracy: 0.7768
Epoch 7/80
112/112 [==============================] - 269s 2s/step - loss: 1.2321 - accuracy: 0.8424 - val_loss: 2.5410 - val_accuracy: 0.7797
Epoch 8/

### 18 Verifying the model
The model is verified by a random image from google.

In [ ]:
img_path = '/content/gdrive/My Drive/Kaggle/Indian currency dataset v1/my_test/500_1.jpg'
img = image.load_img(img_path, target_size=(224, 224))
plt.imshow(img)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
print(model.predict(x))